In [1]:
from keras.layers import Dropout, Dense, GRU, Embedding,LSTM
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import model_selection, naive_bayes, svm
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert text to word embedding (Using Glove Vector 50 Dim):

In [2]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

# Violence Classification based RCNN

In [10]:
def Build_Model_RCNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50):

    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector



    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    
    
    model.add(Dense(128,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [11]:
#Corpus = pd.read_csv(r"aji-Arabic_corpus.csv")

import nltk 
words = set(nltk.corpus.words.words())

Corpus = pd.read_csv(r"Violence in Yemens Social Media.csv",sep=';')




In [12]:
list_term=[]
for term in Corpus.Text:
    sent = " ".join(w for w in nltk.wordpunct_tokenize(term) if w.lower() in words or not w.isalpha())
    list_term.append(sent)
dic={'Text':list_term, 'Dominant_Topic':Corpus.Dominant_Topic}
Corpus=pd.DataFrame(dic)

In [13]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Corpus['Text'],Corpus['Dominant_Topic'],test_size=0.2)


In [14]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)



Found 7163 unique tokens.
(77062, 500)
Total 400000 word vectors.


In [15]:
model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, 5)


model_RCNN.summary()

model_RCNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=15,
                              batch_size=128,
                              verbose=2)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           358200    
                                                                 
 dropout_1 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 499, 256)          25856     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 249, 256)         0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 248, 256)          131328    
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 124, 256)         0         
 1D)                                                  

In [16]:
term_topic0=['violence', 'organization', 'government', 'practice', 'objective', 'social', 'religious', 'state', 'hundred', 'unlawful']
term_topic1=['terrorist', 'militia', 'people', 'yemeni', 'houthi', 'year', 'eman', 'girl', 'village', 'support']
term_topic3=['mine', 'international', 'home', 'plant', 'city', 'displace', 'terror', 'blow', 'threat', 'face']
term_topic4=['group', 'crime', 'life', 'take', 'racist', 'believe', 'call', 'control', 'send', 'carry']
term_topic5=['child', 'kill', 'recruit', 'political', 'houthis', 'civilian', 'war', 'woman', 'force', 'taiz']


In [17]:
Topic0="Religious organization violence"
Topic1="Houthi militias terrorize Yemeni people"
Topic3="international peace threats"
Topic4="Racist Tendencies"
Topic5="Recruitment and killing of children"

In [18]:
predicted = model_RCNN.predict(X_test_Glove)

predicted = np.argmax(predicted, axis=1)

482/482 [==============================] - 108s 216ms/step


In [20]:
from sklearn.metrics import classification_report

target_names = [Topic0, Topic1, Topic3, Topic4, Topic5]
report=classification_report(y_test, predicted, target_names=target_names, digits=2)
print(report)

                                         precision    recall  f1-score   support

        Religious organization violence       0.98      0.73      0.84      1296
Houthi militias terrorize Yemeni people       0.87      0.99      0.93      8402
            international peace threats       0.93      0.82      0.87      1138
                      Racist Tendencies       0.97      0.85      0.91      1928
    Recruitment and killing of children       0.96      0.82      0.89      2649

                               accuracy                           0.91     15413
                              macro avg       0.94      0.84      0.89     15413
                           weighted avg       0.91      0.91      0.91     15413

